<a href="https://colab.research.google.com/github/Charmiece/Final-Project---CreditCard-Fraud/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the packages
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import sklearn.exceptions
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import OneClassSVM
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
from imblearn.under_sampling import NearMiss 
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics, preprocessing, svm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report,accuracy_score
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
from imblearn.metrics import classification_report_imbalanced
from collections import Counter

from sklearn.ensemble import RandomForestClassifier

In [ ]:
# loading data
clean_df = pd.read_csv("https://fraudcreditcard.s3.amazonaws.com/clean_data_train.csv")

In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   merchant    1296675 non-null  object 
 1   category    1296675 non-null  object 
 2   amt         1296675 non-null  float64
 3   gender      1296675 non-null  object 
 4   unix_time   1296675 non-null  int64  
 5   merch_lat   1296675 non-null  float64
 6   merch_long  1296675 non-null  float64
 7   is_fraud    1296675 non-null  int64  
 8   age         1296675 non-null  int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 89.0+ MB


In [ ]:
print(clean_df.shape)

(1296675, 9)


In [ ]:
# taking smaller sample to run the model faster

clean_df= clean_df.sample(frac = 0.1,random_state=1)

print(clean_df.shape)
clean_df.head(10)

(129668, 9)


,merchant,category,amt,gender,unix_time,merch_lat,merch_long,is_fraud,age
94102,fraud_Funk Group,grocery_net,20.00,F,1330158280,46.398331,-99.813959,0,74
198791,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,F,1334260215,45.687331,-117.488135,0,79
1238587,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,F,1370037053,27.254081,-81.974799,0,44
619078,fraud_Conroy-Emard,food_dining,45.38,F,1348160812,34.551957,-83.374265,0,25
573850,fraud_Adams-Barrows,health_fitness,27.52,M,1346455370,43.032957,-83.521294,0,27
1201872,fraud_Effertz LLC,entertainment,131.75,F,1368814877,35.068173,-90.274482,0,45
61984,fraud_Morissette LLC,entertainment,146.69,F,1328478474,38.229734,-94.197224,0,35
552029,fraud_Wiegand-Lowe,grocery_net,74.27,M,1345809559,34.230048,-82.674878,0,80
1079425,fraud_Harris Group,food_dining,11.22,F,1364105244,33.700800,-80.342778,0,61
681734,fraud_Dach-Nader,misc_net,3.96,F,1350531664,29.905143,-98.185725,0,55


In [ ]:
clean_df.isnull().sum()

merchant      0
category      0
amt           0
gender        0
unix_time     0
merch_lat     0
merch_long    0
is_fraud      0
age           0
dtype: int64

In [ ]:
# Create our features
X = pd.get_dummies(clean_df, columns=['merchant', 'category', 'gender'])
                
# Create our target
y = clean_df['is_fraud']

In [ ]:
X.describe()

,amt,unix_time,merch_lat,merch_long,is_fraud,age,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,...,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_F,gender_M
count,129668.000000,1.296680e+05,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,...,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000
mean,70.061919,1.349218e+09,38.532677,-90.195662,0.005437,48.721990,0.001365,0.001504,0.001419,0.001473,...,0.095397,0.086135,0.049311,0.060840,0.070364,0.075238,0.089536,0.031272,0.549758,0.450242
std,153.627239,1.279134e+07,5.094736,13.758655,0.073535,17.374394,0.036921,0.038750,0.037643,0.038351,...,0.293764,0.280565,0.216516,0.239038,0.255761,0.263776,0.285517,0.174053,0.497520,0.497520
min,1.000000,1.325376e+09,19.035472,-166.657834,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.630000,1.338757e+09,34.751272,-96.899931,0.000000,35.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47.360000,1.349244e+09,39.360173,-87.408320,0.000000,47.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,82.940000,1.359235e+09,41.938529,-80.190236,0.000000,60.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,17897.240000,1.371817e+09,66.645176,-66.963975,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Preprocessing with RandomOvrsampler.

In random oversampling, instances of the minority class are randomly selected and added to the training set until the majority and minority classes are balanced.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)
X_train.shape
            

(97251, 715)

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 128963, 1: 128963})

In [ ]:
# Train the  model using the data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[30779,  1462],
       [   38,   138]])

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.8693724605316212

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.95      0.78      0.98      0.87      0.76     32241
          1       0.09      0.78      0.95      0.16      0.87      0.74       176

avg / total       0.99      0.95      0.79      0.97      0.87      0.76     32417



Preprocessing with SMOTE Oversampling.

The synthetic minority oversampling technique (SMOTE) is another oversampling approach to deal with unbalanced datasets. In SMOTE, like random oversampling, the size of the minority is increased.


In [ ]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

Counter({0: 96722, 1: 96722})

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.868922722620266

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[30750,  1491],
       [   38,   138]])

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.95      0.78      0.98      0.86      0.76     32241
          1       0.08      0.78      0.95      0.15      0.86      0.74       176

avg / total       0.99      0.95      0.79      0.97      0.86      0.76     32417



Preprocessing with Undersampling.

Undersampling is another technique to address class imbalance. Undersampling takes the opposite approach of oversampling. Instead of increasing the number of the minority class, the size of the majority class is decreased.

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.868922722620266

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[32241,     0],
       [  176,     0]])

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      1.00      0.00      1.00      0.00      0.00     32241
          1       0.00      0.00      1.00      0.00      0.00      0.00       176

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     32417



Preprocessing with Combination (Over and Under) Sampling.

SMOTEENN combines the SMOTE and Edited Nearest Neighbors (ENN) algorithms. SMOTEENN is a two-step process:
* Oversample the minority class with SMOTE.
* Clean the resulting data with an undersampling strategy. If the two nearest neighbors of a data point belong to two different classes, that data point is dropped.



In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[    0, 32241],
       [    0,   176]])

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00     32241
          1       0.01      1.00      0.00      0.01      0.00      0.00       176

avg / total       0.00      0.01      0.99      0.00      0.00      0.00     32417

